In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

'''
参考：https://github.com/graykode/nlp-tutorial/blob/master/2-1.TextCNN/TextCNN_Torch.ipynb 
https://tangshusen.me/Dive-into-DL-PyTorch/#/chapter10_natural-language-processing/10.8_sentiment-analysis-cnn
'''

'\n参考：https://github.com/graykode/nlp-tutorial/blob/master/2-1.TextCNN/TextCNN_Torch.ipynb \nhttps://tangshusen.me/Dive-into-DL-PyTorch/#/chapter10_natural-language-processing/10.8_sentiment-analysis-cnn\n'

In [2]:
dtype = torch.FloatTensor

# 3 words sentences (=sequence_length is 3)
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

In [3]:
#将每句话切割并转换成相应的index
inputs = []
for sen in sentences:
    inputs.append(np.asarray([word_dict[n] for n in sen.split()]))
print(inputs)

targets = []
for out in labels:
    targets.append(out) # To using Torch Softmax Loss function

input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))

[array([10,  8,  7]), array([ 4,  0, 15]), array([11, 13, 12]), array([10,  5,  7]), array([ 1,  9, 14]), array([2, 6, 3])]


In [4]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, kernel_sizes, num_channels, num_classes, sequence_length):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder = nn.Linear(sum(num_channels), num_classes)
        # 时序最大池化层没有权重，所以可以共用一个实例
        self.pool = nn.MaxPool1d(sequence_length - kernel_sizes[0] + 1)
        self.convs = nn.ModuleList()
        for c,k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels=embedding_size, out_channels=c, kernel_size=k))
        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        embeddings = embeddings.permute(0, 2, 1)
        encoding = torch.cat([self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs], dim=1)
        outputs = self.decoder(self.dropout(encoding))
        return outputs

In [12]:
#TextCNN Parameters
embedding_size = 10
sequence_length = 3
num_classes = 2     #0 or 1
kernel_sizes = [2, 2, 2]
num_channels = [10, 10, 10]

model = TextCNN(vocab_size, embedding_size, kernel_sizes, num_channels, num_classes, sequence_length)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Training
for epoch in range(5000):
    optimizer.zero_grad()
    output = model.forward(input_batch)     # output:[batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)

    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.000201
Epoch: 2000 cost = 0.002537
Epoch: 3000 cost = 0.000286
Epoch: 4000 cost = 0.000007
Epoch: 5000 cost = 0.000000


In [13]:
# Test
test_text = 'this is you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Variable(torch.LongTensor(tests))

# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

this is you is Bad Mean...


In [14]:
''' 完成 '''

' 完成 '